In [8]:
from event_adt import *
from cap_adt import *
from grid_adt import *
from individual_adt import *
from math import log
from random import random,randint,choice
import matplotlib.pyplot as plt

N=10  
Ps=96
Pi=4
Th=50
Td=1
Tr=10
Tm=20
pa=0.4
pd=0.6
pr=0.3
pm=0.5
TaS=1
TaE=1
TaI=10

def Ta(e):
    if e=="S":
        return TaS
    elif e=="E":
        return TaE
    elif e=="I":
        return TaI
    
obs=[]
for i in range(-N,N+1,1): 
    obs=obs + [(2,i)]

def pe(a):
    return 1/(2*log(1.8))-1/(2*log((a*(a-1))/5+1.8))


def exprandom(m): 
    x = random()   
    return -m*log(x)
    

In [25]:
def sim():
    c=CAP()
    g=Grid(N,obs)
    ni=-1
    li=[]
        
    for n1 in range(Pi): 
        i=Ind(n1)
        li=li+[i]
        ni=ni+1 
        i.set_state("I")
        
        ed=Event(n1,exprandom(Td),"des")
        c.addE(ed)
        er=Event(n1,exprandom(Tr),"rep")
        c.addE(er)
        em=Event(n1,exprandom(Tm),"mor")
        c.addE(em)
        ea=Event(n1,exprandom(Ta("I")),"aval") 
        c.addE(ea)
          
    for n2 in range(Pi,Ps+Pi):
        i=Ind(n2)
        li=li+[i]
        ni=ni+1 

        ed=Event(n2,exprandom(Td),"des")
        c.addE(ed)
        er=Event(n2,exprandom(Tr),"rep")
        c.addE(er)
        em=Event(n2,exprandom(Tm),"mor")
        c.addE(em)
        ea=Event(n2,exprandom(Ta("S")),"aval")
        c.addE(ea)
        
     
    Pa=round(pa*(Ps+Pi))  
    if Pa!=0:
    
        la=[]
        for na in range(Pa):
            i=choice(li) 
            list.remove(li,i)
            la=la+[i]
            
            

        ic=choice(la)
        la.remove(ic)
        x=randint(-N,N)
        y=randint(-N,N)
        while not g.freeQ(x,y):
            x=randint(-N,N)
            y=randint(-N,N)
                
        g.addI(ic,x,y)
        
        lpos=g.freePos1(x,y)      
        for i in la:
            pos=choice(lpos)
            while not g.freeQ(pos[0],pos[1]):
                 pos=choice(lpos)
            g.addI(i,pos[0],pos[1])
            lpos=list(set(lpos)|set(g.freePos1(pos[0],pos[1])))
            lpos.remove(pos)
                    
            
         
        for i in li: 

            x=randint(-N,N)
            y=randint(-N,N)
            while not g.freeQ(x,y):
                x=randint(-N,N)
                y=randint(-N,N)
                
            g.addI(i,x,y)
    
    else:
        for i in li:
            x=randint(-N,N)
            y=randint(-N,N)
            while not g.freeQ(x,y):
                x=randint(-N,N)
                y=randint(-N,N)
                
            g.addI(i,x,y)
        
        
    lt=[0]
    lin=[Pi]
    
    ce=c.nextE()
    c.delE()
    cid=ce.id_ind() 
    ct=ce.time() 
    ck=ce.kind()
    x=g.posI(cid)[0][0]
    y=g.posI(cid)[0][1]
    ci=g.ind(x,y)
    alive=g.posI(cid)[1]
    cap_empty=False
    
    f=open("resultados.txt","w")
    f.write(str(N)+"\n")
    f.write(str(obs)+"\n")
    
    while ct<=Th and not cap_empty:
        
        if ck=="des" and alive:
            p=random() 
            if g.numberInf1(x,y)>=3 and len(g.freePos1(x,y))>0:                                 
                ncp=choice(g.freePos1(x,y)) 
                g.addI(ci,ncp[0],ncp[1])    
                g.delI(x,y)   
                                                                             
            elif p<pd and len(g.freePos1(x,y))>0:                                                              
                ncp=choice(g.freePos1(x,y)) 
                g.addI(ci,ncp[0],ncp[1])
                g.delI(x,y)   
                                                     
            ntd=ct+exprandom(Td)
            ned=Event(cid,ntd,"des")
            c.addE(ned)
            
        if ck=="rep" and alive:
            p=random()
            if p<pr and len(g.freePos1(x,y))>1 and g.numberNeigh1(x,y)>0:
                n=ni+1
                i=Ind(n)
                ni=ni+1
                ncp=choice(g.freePos1(x,y))
                g.addI(ci,ncp[0],ncp[1])
                       
                ed=Event(n,ct+exprandom(Td),"des")
                c.addE(ed)
                er=Event(n,ct+exprandom(Tr),"rep")
                c.addE(er)
                em=Event(n,ct+exprandom(Tm),"mor")
                c.addE(em)
                ea=Event(n,ct+exprandom(Ta("S")),"aval")
                c.addE(ea)
                    
                    
            ntr=ct+exprandom(Tr)
            ner=Event(cid,ntr,"rep")
            c.addE(ner)
                       
        if ck=="mor" and alive:
            p=random()
            if ci.state()=="I":
                
                pm1=1.1*pm
                if pm1>1: 
                    pm1=1
                    
                    if p<pm1:
                        g.delI(x,y)    
                         
            elif p<pm:
                g.delI(x,y)
                
                       
        if ck=="aval" and alive:
            p=random()
            
            n1=g.numberInf1(x,y)
            n2=g.numberInf2(x,y)
            a=2*n1+n2
            pea=pe(a)
                                    
            if ci.state()=="S":
                if p<pea and (n1>0 or n2>0):
                    g.delI(x,y)
                    ci.set_state("E")
                    g.addI(ci,x,y) 
                    ea=Event(cid,ct+exprandom(Ta("E")),"aval")
                    c.addE(ea)
                else: 
                    ea=Event(cid,ct+exprandom(Ta("S")),"aval") 
                    c.addE(ea)
                       
            elif ci.state()=="E":
                g.delI(x,y)
                ci.set_state("I")
                g.addI(ci,x,y)
                ea=Event(cid,ct+exprandom(Ta("I")),"aval") 
                c.addE(ea)
                       
            elif ci.state()=="I":
                g.delI(x,y)
                ci.set_state("R")
                g.addI(ci,x,y)
                       
        f.write(str(g.pos_by_state())+"\n")
        
        if alive:
            lt=lt+[ct] 
            lin=lin+[len(g.pos_by_state()[2])]

        
        ce=c.nextE()  
        c.delE()
        cid=ce.id_ind() 
        (p,b)=g.posI(cid)
        while not b:
            ce=c.nextE()  
            c.delE()
            cid=ce.id_ind() 
            (p,b)=g.posI(cid)
        
        ct=ce.time() 
        ck=ce.kind()
        x=g.posI(cid)[0][0]
        y=g.posI(cid)[0][1]
        ci=g.ind(x,y)
        alive=g.posI(cid)[1]
        
        if c.size()==0:
            cap_empty=True
        
    f.close()
    
    
    plt.xlabel('Time')
    plt.ylabel('Number of infected individuals')
    plt.plot(lt,lin,'r-')
    
    